# HW3 - Q3 [35 pts]

## Important Notices

<div class="alert alert-block alert-danger">
    WARNING: <strong>REMOVE</strong> any print statements added to cells with "#export" that are used for debugging purposes befrore submitting because they will crash the autograder in Gradescope. Any additional cells can be used for testing purposes at the bottom. 
</div>

<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> remove any comment that says "#export" because that will crash the autograder in Gradescope. We use this comment to export your code in these cells for grading.
</div>

<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> import any additional libraries into this workbook.
</div>

All instructions, code comments, etc. in this notebook **are part of the assignment instructions**. That is, if there is instructions about completing a task in this notebook, that task is not optional.  

<div class="alert alert-block alert-info">
    You <strong>must</strong> implement the following functions in this notebook to receive credit.
</div>

`user()`

`long_trips()`

`manhattan_trips()`

`weighted_profit()`

`final_output()`

Each method will be auto-graded using different sets of parameters or data, to ensure that values are not hard-coded.  You may assume we will only use your code to work with data from the NYC-TLC dataset during auto-grading.

<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> remove or modify the following utility functions:
</div>

`load_data()`

`main()`

<div class="alert alert-block alert-info">
    Do <strong>not</strong> change the below cell. Run it to initialize your PySpark instance. If you don't get any output, make sure your Notebook's Kernel is set to "PySpark" in the top right corner.
</div>

In [64]:
sc

Calculation started (calculation_id=8ac9563b-40ab-2301-9b4a-34344ecc456b) in (session=34c95625-910a-ed36-367a-215ad3a60528). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.
<SparkContext master=athena appName=default>



<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> remodify the below cell. It contains the function for loading data and all imports, and the function for running your code.
</div>

In [65]:
#export
from pyspark.sql.functions import *
from pyspark.sql import *

Calculation started (calculation_id=08c9563b-45de-dc74-91c3-35cea1c3bbc6) in (session=34c95625-910a-ed36-367a-215ad3a60528). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


In [66]:
#### DO NOT CHANGE ANYTHING IN THIS CELL ####

def load_data(size='small'):
    # Loads the data for this question. Do not change this function.
    # This function should only be called with the parameter 'small' or 'large'
    
    if size != 'small' and size != 'large':
        print("Invalid size parameter provided. Use only 'small' or 'large'.")
        return
    
    input_bucket = "s3://cse6242-hw3-q3"
    
    # Load Trip Data
    trip_path = '/'+size+'/yellow_tripdata*'
    trips = spark.read.csv(input_bucket + trip_path, header=True, inferSchema=True)
    print("Trip Count: ",trips.count()) # Prints # of trips (# of records, as each record is one trip)
    
    # Load Lookup Data
    lookup_path = '/'+size+'/taxi*'
    lookup = spark.read.csv(input_bucket + lookup_path, header=True, inferSchema=True)
    
    return trips, lookup

def main(size, bucket):
    # Runs your functions implemented above.
    
    print(user())
    trips, lookup = load_data(size=size)
    trips = long_trips(trips)
    mtrips = manhattan_trips(trips, lookup)
    wp = weighted_profit(trips, mtrips)
    final = final_output(wp, lookup)
    
    # Outputs the results for you to visually see
    final.show()
    
    # Writes out as a CSV to your bucket.
    final.write.csv(bucket)

Calculation started (calculation_id=dac9563b-4758-a424-f8d4-d8d7cc5d47ad) in (session=34c95625-910a-ed36-367a-215ad3a60528). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


# Implement the below functions for this assignment:
<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> change any function inputs or outputs, and ensure that the dataframes your code returns align with the schema definitions commented in each function. Do <strong>NOT</strong> remove the #export comment from each of the code blocks either. This can prevent your code from being converted to a python file.
</div>

## 3.1 [1 pt] Update the `user()` function
This function should return your GT username, eg: gburdell3

In [67]:
#export
def user():
    # Returns a string consisting of your GT username.
    return 'hpavlovich3'

Calculation started (calculation_id=d6c9563b-4899-3273-baa5-e0aecb76a6f7) in (session=34c95625-910a-ed36-367a-215ad3a60528). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


## 3.2 [2 pts] Update the `long_trips()` function
This function filters trips to keep only trips greater than or equal to 2 miles.

In [68]:
#export
def long_trips(trips):
    # Returns a Dataframe (trips) with Schema the same as :trips:
    trips = trips.filter(trips.trip_distance >=2)
    return trips

Calculation started (calculation_id=68c9563b-49ef-f447-318f-3638879a15f4) in (session=34c95625-910a-ed36-367a-215ad3a60528). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


## 3.3 [6 pts] Update the `manhattan_trips()` function

This function determines the top 20 locations with a `DOLocationID` in manhattan by passenger_count (pcount).

Example output formatting:

```
+--------------+--------+
| DOLocationID | pcount |
+--------------+--------+
|             5|      15|
|            16|      12| 
+--------------+--------+
```

In [69]:
#export
def manhattan_trips(trips, lookup):
        # Returns a Dataframe (mtrips) with Schema: DOLocationID, pcount

    mtrips_hold = trips.join(lookup, col("DOLocationID") == col("LocationID")) \
            .filter(col("Borough") == "Manhattan")

    mtrips = mtrips_hold.groupBy("DOLocationID") \
        .agg(sum("passenger_count").alias("pcount")) \
        .select("DOLocationID", "pcount") \
        .orderBy(desc(col("pcount"))) \
        .limit(20)
    
    return mtrips


Calculation started (calculation_id=9cc9563b-4b0c-acba-452c-93e379c0dbc8) in (session=34c95625-910a-ed36-367a-215ad3a60528). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


## 3.4 [6 pts] Update the `weighted_profit()` function
This function should determine the average `total_amount`, the total count of trips, and the total count of trips ending in the top 20 destinations and return the `weighted_profit` as discussed in the homework document.

Example output formatting:
```
+--------------+-------------------+
| PULocationID |  weighted_profit  |
+--------------+-------------------+
|            18| 33.784444421924436| 
|            12| 21.124577637149223| 
+--------------+-------------------+
```

In [70]:
#export
def weighted_profit(trips, mtrips): 
    # Returns a Dataframe (wp) with Schema: PULocationID, weighted_profit
    # Note: Use decimal datatype for weighted profit (NOTE: DON'T USE FLOAT)
    # Our grader will be only be checking the first 8 characters for each value in the dataframe

    # average costs and total counts
    trip_hold = trips.groupBy("PULocationID")\
        .agg(avg("total_amount").alias("avg_amount"),
                         count("*").alias("total"))

    wp_hold = trips.join(mtrips, "DOLocationID", "inner") \
        .groupBy("PULocationID") \
        .agg(count("*").alias("PU_count"))

    wp = wp_hold.join(trip_hold, 'PULocationID', 'left') \
        .withColumn("weighted_profit", ((col("PU_count") / col("total")) *
                    col("avg_amount")).cast('decimal(24,10)')) \
        .select("PULocationID", "weighted_profit")

    return wp

Calculation started (calculation_id=6cc9563b-4c36-2720-f4fc-a8e8733d7e83) in (session=34c95625-910a-ed36-367a-215ad3a60528). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


## 3.5 [5 pts] Update the `final_output()` function
This function will take the results of `weighted_profit`, links it to the `borough` and `zone` and returns the top 20 locations with the highest `weighted_profit`.

Example output formatting:
```
+------------+---------+-------------------+
|    Zone    | Borough |  weighted_profit  |
+----------------------+-------------------+
| JFK Airport|   Queens|  16.95897820117925|
|     Jamaica|   Queens| 14.879835188762488|
+------------+---------+-------------------+
```

In [71]:
#export
def final_output(wp, lookup): 
    # Returns a Dataframe (final) with Schema: Zone, Borough, weighted_profit
    # Note: Use decimal datatype for weighted profit (NOTE: DON'T USE FLOAT)
    # Our grader will be only be checking the first 8 characters for each value in the dataframe
    
    lookup_hold = lookup.withColumnRenamed("LocationID", "PULocationID")
    wp_1 = wp.join(lookup_hold, "PULocationID", "inner") \
        .select("Zone", "Borough", "weighted_profit")\
        .orderBy(desc(col("weighted_profit"))) \
        .limit(20)
    
    return wp_1

Calculation started (calculation_id=a6c9563b-4d78-4f03-f8c2-d4fd3f5f1c94) in (session=34c95625-910a-ed36-367a-215ad3a60528). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


#### Testing

<div class="alert alert-block alert-info">
    You may use the below cell for any additional testing you need to do, however any code implemented below will not be run or used when grading.
</div>

In [7]:
# trips,lookup = load_data()
# trips.show()
# lookup.show()
# long_trips = long_trips(trips)

In [8]:
# mtrips = manhattan_trips(trips, lookup)
# mtrips.show()

In [9]:
# wp = weighted_profit(trips, mtrips)
# wp.show()

In [10]:
# final = final_output(wp, lookup)
# final.show()

In [72]:
main("large", "s3://cse6242-hpavlovich3/test3")

Calculation started (calculation_id=42c9563b-4ea7-10c4-9cb8-36d293fa32fd) in (session=34c95625-910a-ed36-367a-215ad3a60528). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.
hpavlovich3
Trip Count:  187203269
+--------------------+-------------+---------------+
|                Zone|      Borough|weighted_profit|
+--------------------+-------------+---------------+
|        Baisley Park|       Queens|  29.3604557791|
|Flushing Meadows-...|       Queens|  27.3048457336|
|       South Jamaica|       Queens|  26.2949162399|
|     Randalls Island|    Manhattan|  24.1509899402|
|        Astoria Park|       Queens|  21.7064171121|
|Briarwood/Jamaica...|       Queens|  19.9450646318|
|Springfield Garde...|       Queens|  19.4683092888|
|             Jamaica|       Queens|  19.2839430001|
|              Corona|       Queens|  18.2287692482|
|   LaGuardia Airport|       Queens|  18.1813388084|
|         Jamaica Bay|       Queens|  17.1005294468|
|             Maspeth|       Queens|  17.0054506401|
|Eltingville/Annad...|Staten Island|  16.8377647569|
|         JFK Airport|       Queens|  16.7777253482|
|        Battery Park|    Manhattan|  12.